In [1]:
import numpy as np
import pandas as pd

import optuna

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

from sklearn.preprocessing import OrdinalEncoder
import category_encoders as ce
from sklearn.preprocessing import KBinsDiscretizer


In [5]:
df = pd.read_csv("../../dataset/kaggle_ventilator/train_5folds.csv")
test_df = pd.read_csv("../../dataset/kaggle_ventilator/test.csv")

In [6]:
def add_features(df, transform="robust"):

    df["u_in_log1p"] = np.log1p(df["u_in"])
    df["u_in_power"] = np.power(df["u_in"], 2)
    df["u_in_round2"] = np.round(df["u_in"], 2)

    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    r_map = {5: 0, 20: 1, 50: 2}
    c_map = {10: 0, 20: 1, 50: 2}
    df['R'] = df['R'].map(r_map)
    df['C'] = df['C'].map(c_map)
    
    return df


In [7]:
train = add_features(df)
test = add_features(test_df)

In [8]:
c = ['time_step', 'u_in', 'u_out',
    'u_in_log1p', 'u_in_power', 'u_in_round2', 'area',
    'u_in_cumsum', 'u_in_lag1', 'u_out_lag1', 'u_in_lag_back1',
    'u_out_lag_back1', 'u_in_lag2', 'u_out_lag2', 'u_in_lag_back2',
    'u_out_lag_back2', 'u_in_lag3', 'u_out_lag3', 'u_in_lag_back3',
    'u_out_lag_back3', 'u_in_lag4', 'u_out_lag4', 'u_in_lag_back4',
    'u_out_lag_back4', 'breath_id__u_in__max', 'breath_id__u_out__max',
    'u_in_diff1', 'u_out_diff1', 'u_in_diff2', 'u_out_diff2',
    'breath_id__u_in__diffmax', 'breath_id__u_in__diffmean', 'u_in_diff3',
    'u_out_diff3', 'u_in_diff4', 'u_out_diff4', 'cross', 'cross2'
    ]

In [9]:
RS = RobustScaler()
train[c] = RS.fit_transform(train[c])
test[c] = RS.fit_transform(test[c])

In [10]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,...,u_in_diff2,u_out_diff2,breath_id__u_in__diffmax,breath_id__u_in__diffmean,u_in_diff3,u_out_diff3,u_in_diff4,u_out_diff4,cross,cross2
0,1,1,1,2,-0.989052,-0.937384,-1.0,5.837492,4,-1.100643,...,0.217993,0.0,0.326941,1.273864,0.121028,0.0,0.068315,0.0,0.0,-0.665541
1,2,1,1,2,-0.963608,3.049278,-1.0,5.907794,4,0.878825,...,58.762670,0.0,-0.178436,-1.450205,38.091005,0.0,26.306250,0.0,0.0,-0.665541
2,3,1,1,2,-0.938006,3.948195,-1.0,7.876254,4,1.011274,...,71.696779,0.0,-0.292389,-2.064430,46.652513,0.0,32.222406,0.0,0.0,-0.665541
3,4,1,1,2,-0.912278,4.013452,-1.0,11.742872,4,1.019963,...,14.110409,0.0,-0.300662,-2.109020,47.101127,0.0,32.651890,0.0,0.0,-0.665541
4,5,1,1,2,-0.886409,4.568332,-1.0,12.234987,4,1.089713,...,9.058184,0.0,-0.371002,-2.488167,14.415967,0.0,36.184309,0.0,0.0,-0.665541


In [11]:
train.to_csv("../../dataset/kaggle_ventilator/train_5folds_nb2_robust.csv", index=False)
test.to_csv("../../dataset/kaggle_ventilator/test_nb2_robust.csv", index=False)

In [12]:
sorted(c)

['area',
 'breath_id__u_in__diffmax',
 'breath_id__u_in__diffmean',
 'breath_id__u_in__max',
 'breath_id__u_out__max',
 'cross',
 'cross2',
 'time_step',
 'u_in',
 'u_in_cumsum',
 'u_in_diff1',
 'u_in_diff2',
 'u_in_diff3',
 'u_in_diff4',
 'u_in_lag1',
 'u_in_lag2',
 'u_in_lag3',
 'u_in_lag4',
 'u_in_lag_back1',
 'u_in_lag_back2',
 'u_in_lag_back3',
 'u_in_lag_back4',
 'u_in_log1p',
 'u_in_power',
 'u_in_round2',
 'u_out',
 'u_out_diff1',
 'u_out_diff2',
 'u_out_diff3',
 'u_out_diff4',
 'u_out_lag1',
 'u_out_lag2',
 'u_out_lag3',
 'u_out_lag4',
 'u_out_lag_back1',
 'u_out_lag_back2',
 'u_out_lag_back3',
 'u_out_lag_back4']

In [2]:
train = pd.read_csv("../../dataset/kaggle_ventilator/train_5folds_nb2_robust.csv")

In [3]:
train["u_out"]

0         -1.0
1         -1.0
2         -1.0
3         -1.0
4         -1.0
          ... 
6035995    0.0
6035996    0.0
6035997    0.0
6035998    0.0
6035999    0.0
Name: u_out, Length: 6036000, dtype: float64